# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
#llm_config = {"model": "gpt-3.5-turbo"}
import os
config_list = [{"api_type":"groq","model":"llama3-70b-8192","api_key":os.environ.get("GROQ_API_KEY"),}]
llm_config={"config_list" : config_list}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

{'content': '"Unlocking AI Potential: DeepLearning.AI Revolution"\n\nIn the pursuit of AI excellence, DeepLearning.AI is revolutionizing the field with its cutting-edge solutions. Founded by Andrew Ng, a pioneer in AI, this platform offers a suite of tools and resources to accelerate AI adoption. From novice-friendly courses to advanced frameworks, DeepLearning.AI helps businesses and individuals harness the power of deep learning. With its innovative approach, this platform is bridging the gap between AI technology and real-world applications, empowering a new generation of AI leaders to transform industries and solve complex problems.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

"Revolutionizing AI Education: The DeepLearning.AI Impact"

In a world where AI is transforming industries, accessing quality education is crucial to stay ahead. DeepLearning.AI, founded by Andrew Ng, bridges this gap by providing a platform for learning AI and machine learning. With a focus on hands-on experience, the platform offers a range of courses, tutorials, and projects. From beginners to experts, DeepLearning.AI caters to all, empowering individuals to build and deploy AI models. With its innovative approach, DeepLearning.AI is revolutionizing AI education, making it more accessible and affordable for everyone. Join the AI revolution with DeepLearning.AI!

------------------------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

"Revolutionizing AI Education: The DeepLearning.AI Impact"

In a world where AI is transforming industries, accessing quality education is crucial to stay ahead. DeepLearning.AI, founded by Andrew Ng, bridges this gap by providing a platform for learning AI and machine learning. With a focus on hands-on experience, the platform offers a range of courses, tutorials, and projects. From beginners to experts, DeepLearning.AI caters to all, empowering individuals to build and deploy AI models. With its innovative approach, DeepLearning.AI is revolutionizing AI education, making it more accessible and affordable for everyone. Join the AI revolution with DeepLearning.AI!

------------------------------------------------------------------

## Get the summary

In [15]:
import pprint
pprint.pprint(res.summary)

('Here is a revised version of the blog post, addressing the suggested '
 'improvements:\n'
 '\n'
 '"Democratizing AI Education: The DeepLearning.AI Vision"\n'
 '\n'
 'DeepLearning.AI, founded by Andrew Ng, is on a mission to make AI education '
 'accessible to everyone. By providing a platform for hands-on learning, the '
 'company bridges the gap between AI enthusiasts and professional '
 'opportunities.\n'
 '\n'
 "The platform's unique approach focuses on practical experience, offering a "
 "range of courses, tutorials, and projects. Whether you're a beginner or an "
 'expert, DeepLearning.AI provides the tools and resources needed to build and '
 'deploy AI models. This innovative approach has the potential to '
 'revolutionize the AI industry, making it more inclusive and diverse.\n'
 '\n'
 'With DeepLearning.AI, individuals from all backgrounds can acquire the '
 'skills needed to succeed in AI. By providing access to quality education, '
 'the company is empowering people to bui